# Transfer Learning Task

The goal of this task is to illustrate how pretrained neural networks can be used for soving different classification problems.

## Data

We will use the [Cifar 10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html). It consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

Here are the classes in the dataset, as well as 10 random images from each:

![cifar10](https://raw.githubusercontent.com/mlcollege/rbi/master/specializations/Deep-Learning/src/images/cifar10.png)



In [ ]:
%tensorflow_version 2.x
from tensorflow.python.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape)
print(y_train.shape)

Prepare data

In [ ]:
%tensorflow_version 2.x
from tensorflow.python.keras.utils import np_utils
import numpy as np

n_classes = 10

y_train = np_utils.to_categorical(y_train, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_test -= x_train_mean

Import pretrained model

In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.applications import VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
for layer in base_model.layers[:-4]:
    layer.trainable = False

Add new top layers

In [ ]:
%tensorflow_version 2.x
from tensorflow.python.keras import models, layers
    
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))


Compile the model

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          batch_size = 512, epochs = 20, verbose=1,
          validation_data=(x_test, y_test))

## Evaluate the model

First we need to convert probability vectors to class indices.

In [ ]:
y_pred = model.predict(x_test)

print(y_pred.shape)

In [ ]:
import numpy as np

y_test_class = np.argmax(y_test, axis=1)
y_pred_class = np.argmax(y_pred, axis=1)
print(y_pred_class.shape)

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score


print ("Test accuracy: {:.4f}".format(accuracy_score(y_test_class, y_pred_class)))
print ()
print(metrics.classification_report(y_test_class, y_pred_class, digits=4))

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test_class, y_pred_class))